In [1]:
import pandas as pd
import numpy as np

In [2]:
pnad1 = pd.read_csv("data_pnad1.csv", sep = ',', usecols = [7, 1, 6, 2, 3, 4, 5])
pnad2 = pd.read_csv("data_pnad2.csv", sep = ',', usecols = [7, 1, 6, 2, 3, 4, 5])
aux_emergencial = pd.read_csv("dados_auxilio_sumarizados.csv", sep = ',')

Trata dados auxílio emergencial

In [3]:
aux_emergencial = aux_emergencial.pivot(index = ['mes_disp', 'uf'], columns = 'enquadramento')

In [4]:
aux_emergencial.columns = ['_'.join(col) for col in aux_emergencial.columns.values]

Trata dados pnad1

In [5]:
pnad1 = pnad1[['trimestre_fim', 'uf', 'condicao', 'valor']]
pnad1 = pnad1.pivot(index = ['trimestre_fim', 'uf'], columns = 'condicao')
pnad1.columns = ['_'.join(col) for col in pnad1.columns.values]

In [6]:
pnad1_aux = pnad1.copy().reset_index()

In [7]:
condition = [(pnad1_aux.trimestre_fim == 202003), (pnad1_aux.trimestre_fim == 202006), (pnad1_aux.trimestre_fim == 202009), (pnad1_aux.trimestre_fim == 202012)]
value = [202001, 202004, 202007, 202010]
pnad1_aux['trimestre'] = np.select(condition, value)

In [8]:
pnad1_aux2 = pnad1.copy().reset_index()

In [9]:
condition = [(pnad1_aux2.trimestre_fim == 202003), (pnad1_aux2.trimestre_fim == 202006), (pnad1_aux2.trimestre_fim == 202009), (pnad1_aux2.trimestre_fim == 202012)]
value = [202002, 202005, 202008, 202011]
pnad1_aux2['trimestre'] = np.select(condition, value)

In [10]:
pnad1 = pnad1.reset_index()
pnad1['trimestre'] = pnad1['trimestre_fim']

In [11]:
pnad1_final = pd.concat([pnad1, pnad1_aux, pnad1_aux2])

Trata dados pnad2

In [12]:
pnad2 = pnad2[['trimestre_fim', 'uf', 'condicao', 'valor']]
pnad2 = pnad2.pivot(index = ['trimestre_fim', 'uf'], columns = 'condicao')
pnad2.columns = ['_'.join(col) for col in pnad2.columns.values]

In [13]:
pnad2_aux = pnad2.copy().reset_index()

In [14]:
condition = [(pnad2_aux.trimestre_fim == 202003), (pnad2_aux.trimestre_fim == 202006), (pnad2_aux.trimestre_fim == 202009), (pnad2_aux.trimestre_fim == 202012)]
value = [202001, 202004, 202007, 202010]
pnad2_aux['trimestre'] = np.select(condition, value)

In [15]:
pnad2_aux2 = pnad2.copy().reset_index()

In [16]:
condition = [(pnad2_aux2.trimestre_fim == 202003), (pnad2_aux2.trimestre_fim == 202006), (pnad2_aux2.trimestre_fim == 202009), (pnad2_aux2.trimestre_fim == 202012)]
value = [202002, 202005, 202008, 202011]
pnad2_aux2['trimestre'] = np.select(condition, value)

In [17]:
pnad2 = pnad2.reset_index()
pnad2['trimestre'] = pnad2['trimestre_fim']

In [18]:
pnad2_final = pd.concat([pnad2, pnad2_aux, pnad2_aux2])

Junta os 3 dataframes em um só através do mês e UF

In [19]:
merge1 = pd.merge(aux_emergencial, pnad1_final, how='left', left_on=['mes_disp', 'uf'], right_on=['trimestre', 'uf'])

In [20]:
merge2 = pd.merge(merge1, pnad2_final, how='left', left_on=['trimestre', 'uf'], right_on=['trimestre', 'uf'])

In [21]:
tab_final = merge2.set_index(['trimestre', 'uf'])

In [40]:
tab_final_rnm = tab_final.drop(columns=['trimestre_fim_x', 'trimestre_fim_y']).rename(columns={"sum_BOLSA FAMILIA": "valor_bolsa_familia", "sum_CADUNICO": "valor_cadunico", 
                          "sum_EXTRACAD": "valor_extracad", "count_BOLSA FAMILIA": "qtd_bolsa_familia", "count_CADUNICO": "qtd_cadunico", "count_EXTRACAD": "qtd_extracad", 
                          "valor_Força de trabalho": "total_forca_trabalho", "valor_Força de trabalho - desocupada": "forca_trabalho_descoupada", "valor_Conta própria": "forca_conta_propria", 
                          "valor_Empregado no setor privado, exclusive trabalhador doméstico": "forca_privado", "valor_Empregado no setor público" : "forca_publico", 
                          "valor_Empregador": "forca_empregador", "valor_Trabalhador doméstico": "forca_domestico", "valor_Trabalhador familiar auxiliar": "forca_auxiliar"})

In [41]:
tab_final_rnm.index.names = ['mes_ref', 'uf']

In [44]:
tab_final_rnm.to_csv("aux_pnad_tratada.csv")